# Create dataset 

In [ ]:
# Generate examples of 2 types 
# Very general and useful code
# Run model 

# Please read the source code 
# I put a lot of effort in it 

# Uncomment to get new dataset
# Taked 10 seconds
# ! unzip SDF-JEPA-main.zip
# !python generator.py ../configs/simple_sampler.yaml

# Load dataset 

In [157]:
import pandas as pd 
from ipywidgets import widgets 

In [158]:
df = pd.read_parquet('../data/prepared.parquet')
df.head()

,filename,prefix,middle,suffix,meta,query,output
0,../data/SDF-JEPA-main/evals/main.py,",\n default='configs.yaml')\nparser.add_arg...",)\n\n,import logging\n logging.basicConfig()\...,finish_lines,"<fim_prefix>,\n default='configs.yaml')\npa...",)\n os.environ['MASTER_ADDR'] = 'localhost'...
1,../data/SDF-JEPA-main/src/models/vision_transf...,"cale(layer.mlp.fc2.weight.data, layer_id + 1)\...","(remove)\n """"""\n\n",if masks is not None and not isinstanc...,finish_lines,"<fim_prefix>cale(layer.mlp.fc2.weight.data, la...","and mask tokens\n :return:\n """"""..."
2,../data/SDF-JEPA-main/app/vjepa/train.py,"th=dataset_eval_paths,\n batch_size=ba...",",\n transform=transform,\n","datasets_weights=datasets_weights,\n ...",finish_lines,"<fim_prefix>th=dataset_eval_paths,\n b...","_per_video,\n num_clips_per_clip=num_c..."
3,../data/SDF-JEPA-main/src/models/utils/patch_e...,"roj = nn.Conv2d(in_chans, embed_dim, kernel_si...","(\n self,\n patch_size=16,\n","tubelet_size=2,\n in_chans=3,\n...",finish_lines,"<fim_prefix>roj = nn.Conv2d(in_chans, embed_di...","(self, in_chans, embed_dim):\n super()...."
4,../data/SDF-JEPA-main/src/models/utils/multima...,outs = []\n for m in masks:\n ...,):\n if type(ctxt) is not list:\n ...,if type(tgt) is not list:\n ...,finish_lines,<fim_prefix> outs = []\n for m in mask...,"):\n ctxt = [self.backbone(x, masks=m) ..."


In [159]:
from IPython.display import HTML
import ipywidgets as widgets

def format_text(prefix, middle, suffix):
    # Replace \n with <br> for HTML line breaks
    prefix = prefix.replace('\n', '<br>')
    middle = middle.replace('\n', '<br>')
    suffix = suffix.replace('\n', '<br>')

    # Replace \t with a series of non-breaking spaces for HTML tabs
    prefix = prefix.replace('\t', '&nbsp;' * 4)
    middle = middle.replace('\t', '&nbsp;' * 4)
    suffix = suffix.replace('\t', '&nbsp;' * 4)

    return f"<pre>{prefix}<span style='color:red'>{middle}</span>{suffix}</pre>"

def create_widget(prefix, middle, suffix, output):

    output_true = widgets.Output()
    with output_true:
        display(HTML(format_text(prefix, middle, suffix)))

    output_pred = widgets.Output()
    with output_pred:
        display(HTML(format_text(prefix, output, suffix)))

    # Create titles
    title_true = widgets.HTML("<h3>True</h3>")
    title_pred = widgets.HTML("<h3>Predicted</h3>")

    # Stack titles and outputs vertically
    column_true = widgets.VBox([title_true, output_true])
    column_pred = widgets.VBox([title_pred, output_pred])

    # Add space between columns
    column_true.layout.margin = '0 20px 0 0'  # 20px right margin for the first column

    two_columns = widgets.HBox([column_true, column_pred])

    return two_columns

Let's look 5 random examples from both strategies 

## Finish Lines
Finish lines strategy tries to predict 1-2 lines,  
when user finished typing specific token.  
This could be predicting after "," or "." or ")"  
For the full list and more precise behaviour please read the config at source code

In [160]:
finish_lines_df = df.query('meta == "finish_lines"').sample(5, random_state=123).reset_index().drop('index', axis=1)
finish_lines_df['subjective_score'] = None

In [161]:
widget = create_widget(*finish_lines_df.loc[0, ['prefix', 'middle', 'suffix', 'output']])
display(widget)

### Good: 
- predicted argv correcly 
- started loop correctly

### Bad 
- Didn't take into account that i is compared to len, thus should start enumerate from 1
- Created random function 

### Overall subjective score
3 / 10

In [162]:
finish_lines_df.loc[0, 'subjective_score'] = 3

In [163]:
widget = create_widget(*finish_lines_df.loc[1, ['prefix', 'middle', 'suffix', 'output']])
display(widget)

Unfortunetly not great example, because it's just a comma  
### Good 
- Comma 
### Bad 
- Broken grammar
- Stupid repeated params

### Overall subjective score
1 / 10

In [164]:
finish_lines_df.loc[1, 'subjective_score'] = 1

In [165]:
widget = create_widget(*finish_lines_df.loc[2, ['prefix', 'middle', 'suffix', 'output']])
display(widget)

### Good 
- Correcly finished method header 

### Bad 
- Did nonesence to passed params 

### Overall subjective score 
2 / 10

In [166]:
finish_lines_df.loc[2, 'subjective_score'] = 2

In [167]:
widget = create_widget(*finish_lines_df.loc[3, ['prefix', 'middle', 'suffix', 'output']])
display(widget)

### Good 
- Looks plausable 
- Almost correct

### Bad 
- Wrong and unnecessary second argument 

### Overall subjective score 
4 / 10

In [168]:
finish_lines_df.loc[3, 'subjective_score'] = 4

In [169]:
widget = create_widget(*finish_lines_df.loc[4, ['prefix', 'middle', 'suffix', 'output']])
display(widget)

### Good
- Correctly created argument

### Bad 
- Not what intended 
- Wrong second arg

### Overall subjective score 
3 / 10

In [170]:
finish_lines_df.loc[4, 'subjective_score'] = 3

## Finish Words

Finish words strategy tries to predict a few words after user started typing one word. 

In [171]:
finish_words_df = df.query('meta == "finish_words"').sample(5, random_state=123).reset_index().drop('index', axis=1)

In [172]:
finish_words_df['subjective_score'] = None

In [173]:
widget = create_widget(*finish_words_df.loc[0, ['prefix', 'middle', 'suffix', 'output']])
display(widget)

### Good 
- Basically correct grammar 

### Bad 
- Did nonesence 
  
### Overall subjective score 
2 / 10

In [174]:
finish_words_df.loc[0, 'subjective_score'] = 2

In [175]:
widget = create_widget(*finish_words_df.loc[1, ['prefix', 'middle', 'suffix', 'output']])
display(widget)

### Good
- Nothing 

### Bad 
- Wrong args 
- Wrong grammar  

### Overall subjective score 
2 / 10

In [176]:
finish_words_df.loc[1, 'subjective_score'] = 2

In [177]:
widget = create_widget(*finish_words_df.loc[2, ['prefix', 'middle', 'suffix', 'output']])
display(widget)

### Good 
- Correct

### Bad 
- Slightly more than required 

### Overall subjective score 
9 / 10

In [178]:
finish_words_df.loc[2, 'subjective_score'] = 9

In [179]:
widget = create_widget(*finish_words_df.loc[3, ['prefix', 'middle', 'suffix', 'output']])
display(widget)

### Good 
- Plausable continuation
- Almost correcrt grammar 

### Bad 
- Not correct 

### Overall subjective score 
6 / 10 

In [180]:
finish_words_df.loc[3, 'subjective_score'] = 6

In [181]:
widget = create_widget(*finish_words_df.loc[4, ['prefix', 'middle', 'suffix', 'output']])
display(widget)

### Good 
- Almost plausable 

### Bad 
- Not correct 

### Overall subjective score
4 / 10

In [182]:
finish_words_df.loc[4, 'subjective_score'] = 4

# Metrics 

Hypothesis: 
Predicting after a word was started is much better, than after a comma or any other separator 

In [215]:
from codebleu import calc_codebleu
from nltk.translate.bleu_score import sentence_bleu
from sacrebleu.metrics import BLEU, CHRF, TER
import plotly.express as px
import re

In [216]:
bleu = BLEU()
chrf = CHRF()
ter = TER()

In [185]:
def calc_metrics(df):
    df = df.copy()

    df['e_match'] = (df['output'] == df['middle']).astype(float)

    df['bleu'] = [
        bleu.corpus_score([o], [[m]],).score
        for m, o in zip(df['middle'], df['output'])
    ]

    df['chrf'] = [
        chrf.corpus_score([o], [[m]]).score
        for m, o in zip(df['middle'], df['output'])
    ]

    df['ter'] = [
        ter.corpus_score([o], [[m]]).score
        for m, o in zip(df['middle'], df['output'])
    ]

    df['codebleu'] = [
        calc_codebleu([m], [o], lang='python')['codebleu']
        for m, o in zip(df['middle'], df['output'])
    ]

    return df

In [186]:
df = calc_metrics(df)
df_small = calc_metrics(pd.concat([finish_lines_df, finish_words_df], ignore_index=True))

In [187]:
metrics = ['e_match', 'bleu', 'chrf', 'ter', 'codebleu']

In [188]:
df[metrics].describe()

,e_match,bleu,chrf,ter,codebleu
count,40.000000,40.000000,40.000000,40.000000,40.000000
mean,0.025000,12.155524,26.464994,210.733337,0.284302
std,0.158114,18.017568,19.048590,231.253995,0.174144
min,0.000000,0.000000,2.597403,0.000000,0.046696
25%,0.000000,3.514005,14.311302,92.656250,0.168171
50%,0.000000,6.179192,22.452971,105.555556,0.272017
75%,0.000000,11.577372,34.084185,275.000000,0.333820
max,1.000000,100.000000,100.000000,1300.000000,1.000000


In [189]:
df[metrics].corr().round(2)

,e_match,bleu,chrf,ter,codebleu
e_match,1.00,0.79,0.63,-0.15,0.67
bleu,0.79,1.00,0.89,-0.26,0.72
chrf,0.63,0.89,1.00,-0.32,0.58
ter,-0.15,-0.26,-0.32,1.00,0.05
codebleu,0.67,0.72,0.58,0.05,1.00


All metrics have non 0 values and correlation  
ter as expected has negative correlation with other metrics 

In [190]:
df_show = df.copy()
df_show[metrics] = df_show[metrics] / df_show[metrics].std()
df_show[metrics] -= df_show[metrics].mean()
df_show = df_show[['meta'] + metrics].melt('meta', var_name='metric')

px.box(
    df_show,
    x='metric',
    y='value',
    color='meta'
)

e_match is not good metrics  
bleu, if not take into account one full match, finish_words performed better  
chrf is same, finish_words perform better 
ref is same, finish_words perform better 
codebleu is opposite, finish_lines performed better 

In [191]:
df_show

,meta,metric,value
0,finish_lines,e_match,-0.158114
1,finish_lines,e_match,-0.158114
2,finish_lines,e_match,-0.158114
3,finish_lines,e_match,-0.158114
4,finish_lines,e_match,-0.158114
...,...,...,...
195,finish_words,codebleu,-0.926713
196,finish_words,codebleu,0.042289
197,finish_words,codebleu,0.552085
198,finish_words,codebleu,-0.920685


In [ ]:
metrics.append('subjective_score')
metrics = [m for m in metrics if m != 'e_match']

In [199]:
df_small[metrics].corr().round(2)

,bleu,chrf,ter,codebleu,subjective_score
bleu,1.00,0.82,-0.45,0.40,0.07
chrf,0.82,1.00,-0.52,0.16,0.35
ter,-0.45,-0.52,1.00,0.11,-0.27
codebleu,0.40,0.16,0.11,1.00,0.41
subjective_score,0.07,0.35,-0.27,0.41,1.00


subjective_score has biggest correlation with codebleu  
which makes sence as it is most advanced metrics out of these 

In [211]:
df_show = df_small.copy()
df_show.sort_values(by='subjective_score', inplace=True, ignore_index=True)
df_show.reset_index(inplace=True)

In [213]:
df_show[metrics] = df_show[metrics] / df_show[metrics].std()
df_show[metrics] -= df_show[metrics].mean()
df_show = df_show[['meta', 'index'] + metrics].melt(['meta', 'index'], var_name='metric')

In [214]:
px.scatter(df_show, x='index', y='value', color='metric')

# Conclusion
codebleu looks like the closest metric to subjective score  
Thus, although other metrics say it's better to predict after a few charaters of the word were typed,  
we probably should try to predict after a word was finished 

# Future work

Try more models and other approaches, for example more context, more prediction etc.  
This is more of a toy example showing how such project can be done.  
